In [16]:
import spacy
from spacy.tokens import Span
from policyanalysis.nlptools.correlation import KeywordCorrelator
nlp = spacy.load("en_core_web_sm")
tfhub_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

KeywordCorrelator.add_span_subject_correlator("quick_foxes", ["Brown foxes are quick"])
KeywordCorrelator.add_span_subject_correlator("lazy_animals", ['some animals are lazy'])

In [19]:

threegram_sugg = spacy.pipeline.spancat.build_ngram_suggester([3])
doc = nlp("The quick brown fox jumped over the lazy dog. Another lazy dog went by. What happened next? We do not know.")
n_gram_size = 7

def get_n_gram_tuples(size, doc_len):
    return [(i, i+size) for i in range(doc_len-size+1)]

# tuples = get_n_gram_tuples(n_gram_size, len(doc))
# for t in tuples:
#     s = doc[t[0]:t[1]]
#     print(s._.span_kwd_corr)
print(doc[0:4].text, doc[0:4]._.quick_foxes)
print(doc[4:8].text, doc[4:8]._.lazy_animals)


The quick brown fox 0.6648871
jumped over the lazy 0.361911


In [3]:
from typing import List

In [37]:
def entity_tagger(doc, spans: List, threshold, corr_attr_key, entity_tag):
    good_spans = []
    
    spans = [s for s in spans if s._.get(corr_attr_key) > threshold]
    # for s in spans:
    #     if s._.get(corr_attr_key) > threshold:
    #         good_spans.append(Span(doc, s.start, s.end, label=entity_tag))
    
    # s = spans.pop(0)
    # ent_start = s.start
    # ent_end = s.end
    # while len(spans) > 0:
    #     s = spans.pop(0)
    #     if s.start > ent_end:
    #         good_spans.append(Span(doc, ent_start, ent_end, label=entity_tag))
    #         ent_start = s.start
    #         ent_end = s.end
    #     else:
    #         ent_end = s.end
    # good_spans.append(Span(doc, ent_start, ent_end, label=entity_tag))

    for s in spans:
        try: 
            doc.ents = list(doc.ents) + [Span(doc, s.start, s.end, label=entity_tag)]
        except:
            pass

In [49]:
# doc.ents = list(doc.ents) + [Span(doc, 0, 3, label="GOOD")]
doc.ents = []
n_gram_spans = [doc[t[0]:t[1]] for t in get_n_gram_tuples(4, len(doc))]
n_gram_spans

[The quick brown fox,
 quick brown fox jumped,
 brown fox jumped over,
 fox jumped over the,
 jumped over the lazy,
 over the lazy dog,
 the lazy dog.,
 lazy dog. Another,
 dog. Another lazy,
 . Another lazy dog,
 Another lazy dog went,
 lazy dog went by,
 dog went by.,
 went by. What,
 by. What happened,
 . What happened next,
 What happened next?,
 happened next? We,
 next? We do,
 ? We do not,
 We do not know,
 do not know.]

In [50]:
entity_tagger(doc, n_gram_spans, 0.35, 'quick_foxes', "FAST")
entity_tagger(doc, n_gram_spans, 0.35, 'lazy_animals', "SLOW")

In [51]:
[print(e.text, e.label_, e._.quick_foxes, e._.lazy_animals)  for e in doc.ents]

The quick brown fox FAST 0.6648871 0.16049126
jumped over the lazy SLOW 0.039034665 0.361911
dog. Another lazy SLOW 0.15132414 0.5753756


[None, None, None]

In [61]:
from spacy import displacy

displacy.render(doc[3:], style="ent")
displacy.render(doc, style='ent')